## UNT MODELS for DARPA Hurdle 3 

Given that a Random Guess does not perform well enough to pass the 3rd hurdle of the DARPA Sharing Spectrum Competition, Then a more intelligent solution is required. In this document I propose 3 posible solutions for this 3rd hurdle. 




#### Solution 1: Generic Random Forest Predictor given the past behaviour

   This is the fastest and one of the easiest methods I could think of. Given the last outputs of the Player and the DARPA player let a Random Forest predict the next output given a sequence of previous outputs. 
   
   Input sequences may be M-length or Fixed-Length (e.g. 100 samples) 
     
   Output of the estimator is a M-length-1 vector with the probabilities of each posible output. From here we select the most Probable output as our prediction and the Least probable output as out selection 
   


In [1]:
# import UNT_hurdle3.py  for training and simulation 

import UNT_hurdle3 as h3 
import pandas as pd 
import numpy as np
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score



class RFPlayer:
    ''' A Simple class that simulate a RandomForest base Player for the #3 Hurdle of the DARPA Competition  '''
    import numpy as np
    from sklearn.ensemble import RandomForestClassifier                       
        
        
        
    def __init__ (self, N,retrain=500, seq_size = 100):
        self._type = "RF"
        self.N = N                        ## Random N-states (NOT USED)
        self.M = N                        ##  M-outputs
        self.clf = RandomForestClassifier(n_estimators=25) ## Our estimator
        self.retrain = retrain            ## Retrain after #retrain turns of the game
        self.input_size = seq_size        ## How many lags should we include
        self.Dataset = self.createDataset(seq_size)
        self.record = pd.DataFrame( columns=['Player_Last','Last','pred'])
        self.feature = []
        self.turn = 0
        self.LastPred = -1
                            
        
        # In order to get A -> choice from Q column
    def start(self):

        D,P = [np.random.choice(range(1,self.M)),np.random.choice(range(1,self.M))]
        self.Prev_D = D
        return D,P
        
    def step(self, A):
        
        self.turn = self.turn +1
        self.appendData(A,self.Prev_D, self.LastPred)
        
        if self.turn < self.retrain:
            D,P = self.start()
                    
        else:
            # check if is time to train 
            if self.turn%self.retrain == 0:
                # Train
                self.Train()
                    
            D,P = self.predict()            
                 
        self.Prev_D = D
        self.LastPred = P
        return [D,P]
    
    
    def createDataset(self, lags):
        Player_names = ['Player_lag_{0} '.format(s) for s in range(1,lags)]
        Bot_names = ["Bot_lag_{0}".format(s) for s in range(1,lags)]
        cols = np.concatenate([['Player_Last','Last'],Player_names,Bot_names], axis=0)
        
                                           
        df = pd.DataFrame(columns=['Player_Last','Last'], dtype = 'int32')
        
        return df        
    
    def FormatData(self):
        
        tdf = pd.concat([self.record.Player_Last.shift(n) for n in range(1,self.input_size)], axis=1)
        tdf.columns = ['Player_lag_{0} '.format(s) for s in range(1,self.input_size)]
        ttdf = pd.concat([self.record.Last.shift(n) for n in range(1,self.input_size) ], axis=1)
        ttdf.columns = ['Bot_lag_{0} '.format(s) for s in range(1,self.input_size)]

        df = pd.concat([self.record,tdf,ttdf], axis=1)
        
        return df
    
    def appendData(self, A, D,P):
                
        self.record = self.record.append(pd.DataFrame([[D,A,P]], columns=['Player_Last','Last','pred']),ignore_index=True)
        self.Dataset = self.FormatData()
        
    def Train(self):                
        
        print "Training.. "
        df = self.Dataset.drop('pred',1).copy()
        df['Label'] = self.Dataset.Last.shift(-1)  
        df = df.dropna(axis=0) # Remove NA rows 
        rf = RandomForestClassifier(n_estimators=20) # Basic Clf
        
        #run grid search
        kf = KFold(df.shape[0],n_folds=3)
        grid_search = GridSearchCV(rf,{})  
        
        
        x = df.drop('Label',1)
        y = df.Label
        self.clf = grid_search.fit(x,y)            
        
        #print "Train ACC:", accuracy_score(y, self.clf.predict(x))
        return True    

    
    def predict(self):
        proba = self.clf.predict_proba(self.Dataset.drop('pred',1)[-1:])
        
        l = self.Dataset.Last.unique()
        l.sort(axis=0)
        

        
        return l[proba.argmin()], l[proba.argmax()] # P = More Likely, D = Less Likely 
        
    def restart(self):
        print "Restarting.. "
        self.__init__(self.N,self.retrain,self.input_size)
        return True
        
        
        
        
        

In [2]:
test = RFPlayer(10)
test.start()
test.step(1)
test.step(2)
test.step(3)
test.step(4)
test.step(5)

[4, 7]

In [3]:
for i in range(0,100):
    test.step(1)
    test.step(2)
    test.step(3)
    test.step(4)
    test.step(5)

Training.. 


In [ ]:
#test.clf.predict_proba(test.Dataset[-1:])
test.Dataset.head()

In [ ]:
P1 = h3.RandomPlayer(10,10)
RFP = RFPlayer(10,retrain=29000)

test = h3.Hurdle_MC(P1,RFP,total_turns = 30000)

print "Total MC Runs", test.mc_runs
print "Total Turns", test.turns
%time test.simulate()

Total MC Runs 10
Total Turns 30000
Restarting.. 


In [ ]:
test.PlayerA.Dataset[test.PlayerA.Dataset.Player_Last == test.PlayerA.Dataset.Last]

In [ ]:
test.PlayerA.Dataset.to_csv('Test_RF.csv',index=False)

#### Solution 2: eXtreme Gradient Bosting Machine (XGB)  given the past behaviour

   This is an evolution of the first method, However it requires the instalation of XGB in the docker_container. Same procedure as the Solution 1 
   
   Input sequences may be M-length or Fixed-Length (e.g. 100 samples) 
     
   Output of the estimator is a M-length-1 vector with the probabilities of each posible output. From here we select the most Probable output as our prediction and the Least probable output as out selection 

#### Solution 3: Long Short Term Memory Neural Networks on Tensorflow (The good stuff!)  given the past behaviour

   This is completly different approach from the previous models. (TODO: Add more info for LSTM if needed) 
   
           
   Output of the estimator is a M-length-1 vector with the probabilities of each posible output. From here we select the most Probable output as our prediction and the Least probable output as out selection 
      

In [ ]:
print test.PlayerA.clf.predict_proba(test.PlayerA.Dataset.drop('pred',1)[-5:])
print test.PlayerA.clf.predict(test.PlayerA.Dataset.drop('pred',1)[-5:])


In [ ]:
l = test.PlayerA.Dataset.Last.unique()
l.sort(axis=0)
print l

proba = test.PlayerA.clf.predict_proba(test.PlayerA.Dataset.drop('pred',1)[-1:])
pred = test.PlayerA.clf.predict(test.PlayerA.Dataset.drop('pred',1)[-1:])

print "More Likely: ", l[proba.argmax()], " Pred: ", pred
print "Less Likely: ", l[proba.argmin()], " Pred: ", pred, " Probas: ", proba

In [ ]:
proba.argmax()